# Ann Arbor Load Profiles Construction

To-Do: 
- &check; Load 2018 & 2019 DTE load profiles, which live in raw_csv and are broken down by class
- &check; Collect rate names according to customer class (Res & Com)
- &check; Group together all the residential rates into one hourly profile
- &check; Do the same with commercial rates
- &check; Obtain 2018 & 2019 EIA MI sales by utility and extract Res & Com customer counts from DTE (both Bundled & Delivery)
- &check; Obtain 2019 & 2019 ACS Ann Arbor counts of Res units & Com establishments
- scale DTE Res & Com load profiles down by the ratio of ACS/EIA counts for their respective types
- Upload resulting load profile & plot to Teams 

In [55]:
# import libraries and helper functions
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from helper_functions import make_timestamp, hour_from_timestamp, convert_energy_number_to_int

In [77]:
# customer counts from DTE & A2
    # DTE numbers from eia-861 (raw_xlsx)
    # A2 numbers are currently a guess; waiting on ACS numbers from Eli
dteResCount2018 = 1992276 + 35 # bundled + delivery
dteComCount2018 = 203576 + 4418 
dteResCount2019 = 2003620 + 33
dteComCount2019 = 204660 + 4341

a2ResCount2018 = 151576
a2ComCount2018 = 8222
a2ResCount2019 = 152272
a2ComCount2019 = 8222

In [79]:
# list rates and their type 
# rate types come from DTE 2022 rate book
# may need to change these 
# D1.1, D1.8, D5, D9 secondary -> com
# D1.1, D1.8, D5, D9 res -> res
res_rates = [
    'D1'
    , 'D1.2'
    , 'D1.5'
    , 'D1.6'
    , 'D1.7'
    , 'D1.9'
    , 'D1.9(FL)'
    , 'D2'
    ]
com_rates = [
    'D10'
    , 'D11'
    , 'D4'
    , 'D6.2'
    , 'D8'
    , 'E1'
    , 'E2'
    , 'R3'
    , 'R3(SP)'
    , 'R8'
    , 'R8A'
]


In [181]:
# Load the data and give it a timestamp column rather than Y, M, D, H
year = '2018'
dfDte = pd.read_csv(f'raw_csv/dte-{year}-by-class.csv')
dfDte.insert(
    loc=0
    , column='Timestamp'
    , value=dfDte.apply(lambda row: make_timestamp(int(year), row['Month'], row['Day'], row['Hour']), axis=1)
)
dfDte = dfDte.drop(['Hour', 'Month', 'Day'], axis=1)
dfDte.insert(
        loc=1
        , column='Hour'
        , value=dfDte.Timestamp.apply(hour_from_timestamp)
    )
dfDte[f'{year} kWh'] = dfDte[f'{year} kWh'].apply(convert_energy_number_to_int)

In [182]:
# Group together buckets & add the overlapping rates
dfDteRes = dfDte.query(f"Rate in {res_rates}")
dfDteCom = dfDte.query(f"Rate in {com_rates}")
for rate in ['D1.1', 'D1.8', 'D5', 'D9']: 
    toAddRes = dfDte[(dfDte['Rate'] == rate) & (dfDte['Class'] == 'Residential')]
    toAddCom = dfDte[(dfDte['Rate'] == rate) & (dfDte['Class'] == 'Secondary')]
    dfDteRes = pd.concat([dfDteRes, toAddRes], axis=0)
    dfDteCom = pd.concat([dfDteCom, toAddCom], axis=0)
dfDteRes

,Timestamp,Hour,Class,Rate,2018 kWh
0,2018-01-01 00:00:00,1,Residential,D1,1997548.0
1,2018-01-01 01:00:00,2,Residential,D1,1876672.0
2,2018-01-01 02:00:00,3,Residential,D1,1765548.0
3,2018-01-01 03:00:00,4,Residential,D1,1708013.0
4,2018-01-01 04:00:00,5,Residential,D1,1678923.0
...,...,...,...,...,...
234175,2018-12-31 19:00:00,8756,Residential,D9,1742.0
234177,2018-12-31 20:00:00,8757,Residential,D9,1742.0
234179,2018-12-31 21:00:00,8758,Residential,D9,1742.0
234181,2018-12-31 22:00:00,8759,Residential,D9,1742.0


In [183]:
# aggregate the rates together for res & com. save to csvs
resCom = {'Res': dfDteRes, 'Com': dfDteCom}
timestamps = dfDte.groupby(by='Hour')[['Timestamp']].median() 
for key, val in resCom.items(): 
    resCom[key] = resCom[key].groupby(by='Hour')[[f'{year} kWh']].sum()
    resCom[key].reset_index().set_index(timestamps.Timestamp, inplace=True)
    resCom[key].rename(columns={f'{year} kWh': f'DTE {key} kWh'}, inplace=True)
df = pd.concat(resCom.values(), axis=1)
df.insert(
        loc=0
        , column='Timestamp'
        , value=timestamps
    )
df.to_csv(f'ann-arbor-data/dte-res-com-{year}.csv')

In [211]:
# load resulting csvs and scale their values by ratios defined by the counts above
df2018, df2019 = [pd.read_csv(f'ann-arbor-data/dte-res-com-{year}.csv') 
                  for year in ['2018', '2019']
                ]
def fill_missing_hour(df): 
  df.loc[-1, 'Hour'] = 1635
  df = df.sort_values('Hour').reset_index(drop=True)
  df = df.interpolate()
  return df

df2018, df2019 = list(map(fill_missing_hour, [df2018, df2019]))
df2018.set_index('Timestamp', inplace=True)
df2019.set_index('Timestamp', inplace=True)
ratios = {
    '2018': {
      'Res': a2ResCount2018 / dteResCount2018
      , 'Com': a2ComCount2018 / dteComCount2018
    }
    , '2019': {
      'Res': a2ResCount2019 / dteResCount2019
      , 'Com': a2ComCount2019 / dteComCount2019
    }
}
for type in ['Res', 'Com']: 
  df2018[f'Ann Arbor {type} kWh'] = df2018[f'DTE {type} kWh'] * ratios['2018'][type]
  df2019[f'Ann Arbor {type} kWh'] = df2019[f'DTE {type} kWh'] * ratios['2019'][type]

df = pd.concat([df2018, df2019], axis=0)
df.to_csv('ann-arbor-data/2018-19-a2-res&com-load-profile.csv')